**BİLGİ KAZANIMI**

* X = { a,a,a,a,b,b,b,b,b,b,b}

* a = 4 adet, b = 7 adet

Entropi = 4/11.log(4/11) + 7/11.log(7/11)

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("C:\\Users\\rbesli\\Desktop\\13. Ders\\train.csv")

In [4]:
df.head()

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


In [7]:
import math

X = df.drop("Target", axis=1)
y = df["Target"]

In [10]:
def entropy(y):
    values, counts = np.unique(y, return_counts =True)
    probs = counts / counts.sum()
    return -np.sum(probs * np.log2(probs))

def gini(y):
    values, counts = np.unique(y, return_counts=True)
    probs = counts /counts.sum()
    return 1 - np.sum(probs **2)

print("Entropi Değeri : ", entropy(y))
print("Gini değeri : ", gini(y))

Entropi Değeri :  1.498497439724364
Gini değeri :  0.6277586455288842


In [19]:
def best_split(X, y, criterion="gini"):
    best_feature = None
    best_threshold = None
    best_impurity = float("inf")  # Başlangıçta çok büyük bir değer
 
    # Kriteri seç
    impurity_func = gini if criterion == "gini" else entropy
 
    for feature in X.columns:
        unique_values = X[feature].unique()
 
        for value in unique_values:
            # Sayısal mı, kategorik mi kontrolü
            if np.issubdtype(X[feature].dtype, np.number):
                left_mask = (X[feature] <= value)
                right_mask = ~left_mask
            else:
                left_mask = (X[feature] == value)
                right_mask = ~left_mask  # sol tarafta olmayanlar sağa
 
            if left_mask.sum() == 0 or right_mask.sum() == 0:
                continue
 
            # Sol ve sağ bölümlerin impurity hesaplaması
            left_impurity = impurity_func(y[left_mask])
            right_impurity = impurity_func(y[right_mask])
 
            # Ağırlıklı impurity
            weighted_impurity = (left_mask.sum() / len(y)) * left_impurity + \
                                (right_mask.sum() / len(y)) * right_impurity
 
            # Daha iyi impurity bulunduysa güncelle
            if weighted_impurity < best_impurity:
                best_feature = feature
                best_threshold = value
                best_impurity = weighted_impurity
 
    return best_feature, best_threshold, best_impurity

Seçilen 5 özelliğe göre en iyi split'i bulma

In [22]:
# Seçilen özellikler
selected_features = [
    "Curricular units 2nd sem (grade)",
    "Curricular units 1st sem (grade)",
    "Previous qualification (grade)",
    "Admission grade",
    "Age at enrollment"
]
 
# X_selected oluşturma
X_selected = X[selected_features]
 
# best_split fonksiyonunu kullanarak en iyi bölünmeyi bulma
best_feature, best_threshold, best_impurity = best_split(X_selected, y, criterion="gini")
 
# Sonuçları yazdırma
print("En iyi özellik:", best_feature)
print("Eşik değeri:", best_threshold)
print("Impurity:", best_impurity)

En iyi özellik: Curricular units 2nd sem (grade)
Eşik değeri: 10.5
Impurity: 0.43577337913793224


**Karar Ağacı Node Sınıfı**

In [33]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None)->None:
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value #yaprak tahmini


Ağaç Kurulumu (Recursive)

In [34]:
def build_tree(X, y, depth = 0, max_depth = 3, citerion="gini"):
    if len(np.unique(y)) == 1 or depth >= max_depth:
        values, counts = np.unique(y, return_counts=True)
        return Node(value=values[np.argmax(counts)])
    feature, threshold = best_split(X, y, criterion=criterion)
    if feature is None:
        values, counts = np.unique(y, return_counts=True)
        return Node(value=values[np.argmax(counts)])
    if X[feature].dtype == 'object':
        left_mask = (X[feature] == threshold)
    else : 
        left_mask = (X[feature] <= threshold)
    right_mask =  ~left_mask

#recursive çağrı
    left = build_tree(X[left_mask], y[left_mask], depth+1, max_depth, criterion)
    right = build_tree(X[right_mask], y[right_mask], depth+1, max_depth, criterion)

    return Node(feature, threshold, left, right)

In [35]:
import matplotlib.pyplot as plt 
import networkx as next

def plot_tree(node, graph=None, parent=None, edge_label=""):
    if graph is None:
        graph = nx.DiGraph()
    if node.value is not None:
        label=f"Leaf:{node.value}"
    else:
        label=f"{node.feature} <= {node.threshold}"
    graph.add_node(id(node), label=label)

    if parent: 
        graph.add_edge(parent, id(node), label=edge_label)
    if node.left:
        plot_tree(node.left, graph, id(node), "Yes")
    if node.right:
        plot_tree(node.right, graph, id(node), "No")
    return graph

def draw_graph(graph):
    pos = nx.spring_layout(graph)
    labels = nx.get_node_attributes(graph, "label")
    edge_labels = nx.get_edge_attributes(graph, "label")

    nx.draw(graph, pos, with_labels=False, node_size =2000, node_color="lightblue")
    nx.draw_networkx_labels(graph, pos, labels)
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)
    plt.show()

In [39]:
tree = build_tree(X_selected, y, max_depth=3, criterion="gini")
graph = plot_tree(tree)
draw_graph(graph)


TypeError: build_tree() got an unexpected keyword argument 'criterion'. Did you mean 'citerion'?